In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import functools
import tensorflow as tf
import tensorflow_datasets as tfds
import imageio

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example image

In [ ]:
examples = ['kate', 'library', 'vase', 'vase2']
example_id = 2
data_path = os.path.join('notebooks', 'data', 'inpainting')
image = imageio.imread(os.path.join(data_path, f'{examples[example_id]}.png'))
mask = imageio.imread(os.path.join(data_path, f'{examples[example_id]}_mask.png'))

image = datasets.from_255_to_1_range(
            datasets.to_float32(
                tf.constant(image)))[None,...]
mask = tf.broadcast_to(
            datasets.from_255_to_1_range(
                datasets.to_float32(
                    tf.constant(mask)))[None,...,None],
               shape=tf.shape(image))

image_masked = mask * image

## Simple for comparison

In [ ]:
import skimage.restoration

inpainted_skimage = skimage.restoration.inpaint_biharmonic(image[0].numpy(), tf.cast(mask[0] == False, tf.uint8).numpy())[None,...]
inpainted_border = dppp.inpaint_border(image_masked, mask)

## Half-Quadratic Splitting

In [ ]:
# Load the denoiser
denoiser, (_, denoiser_stddev) = dppp.load_denoiser('models/drugan+_0.0-0.2.h5')

# Callbacks: Print the PSNR every 5th step
callbacks = [ dppp.callback_print_psnr('psnr', 5, image) ]

reconstructed = dppp.hqs_inpaint(image_masked, mask, denoiser, denoiser_stddev, callbacks=callbacks)

In [ ]:
visualize.draw_images([image_masked[0], inpainted_skimage[0], inpainted_border[0], reconstructed[0]], figsize=(50,20))